In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # split되 최대 chunk 크기
    chunk_overlap=200,  # 앞/뒤 문서가 겹쳐지는 부분(연결성을 위해)
)

loader = Docx2txtLoader('./tax_doc.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from langchain_core.messages import AIMessage
from langchain_ollama import ChatOllama

# model = "llama3.2:latest"
model = "llama3.1:latest"

llm = ChatOllama(
    model=model,   # 사용할 언어 모델 지정
    temperature=0.8,    #  상상력   
    # other params...
    num_predict=256, # 생성할 수 있는 최대 토큰 수 
    format="json"
)

In [ ]:
from langchain_ollama import

In [3]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model=model,
)

In [4]:
from langchain_chroma import Chroma

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embeddings)

### 3. 답변 생성을 위한 Retrieval

- `Chroma`에 저장한 데이터를 유사도 검색(`similarity_search()`)를 활용해서 가져옴

In [5]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
retrieved_docs = database.similarity_search(query, k=3)

In [7]:
print(retrieved_docs[1])

page_content='제7조(장기보유 특별공제액의 계산에 관한 적용례) 제95조제5항 및 제6항의 개정규정은 2025년 1월 1일 이후 자산을 양도하는 경우부터 적용한다.

제8조(양도소득의 필요경비 계산 특례에 관한 적용례) 제97조의2제1항의 개정규정은 이 법 시행 이후 자산을 양도하는 경우부터 적용한다.

제9조(조세조약상 비과세ㆍ면제 또는 제한세율의 적용을 위한 경정청구기간에 관한 적용례) 제156조의2제5항 본문, 제156조의4제2항 본문 및 제156조의6제4항 본문의 개정규정은 이 법 시행 당시 각각 같은 개정규정에 따른 경정청구기간이 만료되지 아니한 경우에도 적용한다.

제10조(외국인 통합계좌 원천징수 특례에 관한 적용례) 제156조의9의 개정규정은 이 법 시행 이후 외국인 통합계좌의 명의인에게 국내원천소득을 지급하는 경우부터 적용한다.

제11조(국외 주식매수선택권등 거래명세서 제출에 관한 적용례) 제164조의5의 개정규정은 이 법 시행 이후 임원등이 주식매수선택권을 행사하거나 주식기준보상을 지급받는 경우부터 적용한다.

제12조(신탁소득에 대한 소득세 과세에 관한 경과조치) 이 법 시행 전에 신탁재산에 귀속된 소득에 대해서는 제2조의3제2항의 개정규정에도 불구하고 종전의 규정에 따른다.

제13조(분리과세 기준금액 상향에 관한 경과조치) 이 법 시행 전에 지급받은 연금소득의 분리과세 기준금액에 관하여는 제14조제3항제9호다목의 개정규정에도 불구하고 종전의 규정에 따른다.

제14조(배당가산율 인하에 관한 경과조치) 이 법 시행 전에 지급받은 배당소득의 소득금액 계산에 관하여는 제17조제3항 각 호 외의 부분 단서의 개정규정에도 불구하고 종전의 규정에 따른다.

제15조(주택의 보증금등에 대한 총수입금액 계산의 특례에 관한 경과조치) 이 법 시행 전에 개시한 과세기간의 총수입금액 계산에 관하여는 제25조제1항제2호의 개정규정에도 불구하고 종전의 규정에 따른다.

제16조(장기주택저당차입금 이자 상환액의 소득공제에 관한 경과조치 등) ① 이 

### 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [8]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\Admin\anaconda3\envs\tftest\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [10]:
ai_message = qa_chain({"query": query})

C:\Users\Admin\AppData\Local\Temp\ipykernel_16048\3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [11]:
# 강의에서는 위처럼 진행하지만 업데이트된 LangChain 문법은 `.invoke()` 활용을 권장
ai_message = qa_chain.invoke({"query": query})

In [12]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '{"제162조의3은 현금영수증 가맹점의 입장에서 현금영수증 발급이 의무화되어 있음을 표시하는 것에 대한 법적 근거입니다. 따라서 이 조문은 현금영수증을 발급해야 하는 사업자의 의무와 관련된 것입니다." \n\n  \n\n\n\n\n\n \n\n\n\n\n\n\n\n  \n\n\n\n\n\n  \n\n\n\n\n\n \n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n\n  \n\n \n\n \n\n \n\n\n\n \n\n\n\n \n\n\n\n\n\n\n \n\n'}

# Chroma DB를 이용한 RAG 구성하기

- create_retrieval_chain 사용하기
- create_retrieval_chain의 목적
   1. Retriever를 사용해 문서를 검색.
   2. Combine chain을 통해 검색된 문서를 결합하여 최종 결과를 생성.

-  LangChain에서 create_retrieval_chain를 사용하면 검색과 결합 과정을 유기적으로 연결하여 검색 기반 질문 답변 시스템을 쉽게 구축할 수 있다

```py
%pip install python-dotenv langchain langchain-openai langchain-community langchain-text-splitters docx2txt langchain-chroma
```

In [14]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_doc_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

모델 Embedding 하기 

In [15]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model=model,
)

- chroma DB 생성하기

In [16]:
from langchain_chroma import Chroma

# 이미 저장된 데이터를 사용할 때 
database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embeddings)

In [17]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# `k` 값을 조절해서 얼마나 많은 데이터를 불러올지 결정
# retrieved_docs = database.similarity_search(query, k=3)
retriever = database.as_retriever(search_kwargs={"k": 1},)

In [18]:
retriever.invoke(query)

[Document(metadata={'source': './tax_doc.docx'}, page_content='제7조(장기보유 특별공제액의 계산에 관한 적용례) 제95조제5항 및 제6항의 개정규정은 2025년 1월 1일 이후 자산을 양도하는 경우부터 적용한다.\n\n제8조(양도소득의 필요경비 계산 특례에 관한 적용례) 제97조의2제1항의 개정규정은 이 법 시행 이후 자산을 양도하는 경우부터 적용한다.\n\n제9조(조세조약상 비과세ㆍ면제 또는 제한세율의 적용을 위한 경정청구기간에 관한 적용례) 제156조의2제5항 본문, 제156조의4제2항 본문 및 제156조의6제4항 본문의 개정규정은 이 법 시행 당시 각각 같은 개정규정에 따른 경정청구기간이 만료되지 아니한 경우에도 적용한다.\n\n제10조(외국인 통합계좌 원천징수 특례에 관한 적용례) 제156조의9의 개정규정은 이 법 시행 이후 외국인 통합계좌의 명의인에게 국내원천소득을 지급하는 경우부터 적용한다.\n\n제11조(국외 주식매수선택권등 거래명세서 제출에 관한 적용례) 제164조의5의 개정규정은 이 법 시행 이후 임원등이 주식매수선택권을 행사하거나 주식기준보상을 지급받는 경우부터 적용한다.\n\n제12조(신탁소득에 대한 소득세 과세에 관한 경과조치) 이 법 시행 전에 신탁재산에 귀속된 소득에 대해서는 제2조의3제2항의 개정규정에도 불구하고 종전의 규정에 따른다.\n\n제13조(분리과세 기준금액 상향에 관한 경과조치) 이 법 시행 전에 지급받은 연금소득의 분리과세 기준금액에 관하여는 제14조제3항제9호다목의 개정규정에도 불구하고 종전의 규정에 따른다.\n\n제14조(배당가산율 인하에 관한 경과조치) 이 법 시행 전에 지급받은 배당소득의 소득금액 계산에 관하여는 제17조제3항 각 호 외의 부분 단서의 개정규정에도 불구하고 종전의 규정에 따른다.\n\n제15조(주택의 보증금등에 대한 총수입금액 계산의 특례에 관한 경과조치) 이 법 시행 전에 개시한 과세기간의 총수입금액 계산에 관하여는 제25조제1항제2호의 개정규정에

### 4. Augmentation을 위한 Prompt 활용

- Retrieval된 데이터는 LangChain에서 제공하는 프롬프트(`"rlm/rag-prompt"`) 사용

In [19]:
from langchain_core.messages import AIMessage
from langchain_ollama import ChatOllama

# model = "llama3.2:latest"
model = "llama3.1:latest"

llm = ChatOllama(
    model=model,   # 사용할 언어 모델 지정
    temperature=0.8,    #  상상력   
    # other params...
    num_predict=256, # 생성할 수 있는 최대 토큰 수 
    format="json"
)

In [20]:
from langchain import hub

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")


c:\Users\Admin\anaconda3\envs\tftest\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


### 5. 답변 생성

- [RetrievalQA](https://docs.smith.langchain.com/old/cookbook/hub-examples/retrieval-qa-chain)를 통해 LLM에 전달

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

combine_docs_chain = create_stuff_documents_chain(
    llm, retrieval_qa_chat_prompt
)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [22]:
ai_message = retrieval_chain.invoke({"input": query})

In [23]:
ai_message

{'input': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'context': [Document(metadata={'source': './tax_doc.docx'}, page_content='제7조(장기보유 특별공제액의 계산에 관한 적용례) 제95조제5항 및 제6항의 개정규정은 2025년 1월 1일 이후 자산을 양도하는 경우부터 적용한다.\n\n제8조(양도소득의 필요경비 계산 특례에 관한 적용례) 제97조의2제1항의 개정규정은 이 법 시행 이후 자산을 양도하는 경우부터 적용한다.\n\n제9조(조세조약상 비과세ㆍ면제 또는 제한세율의 적용을 위한 경정청구기간에 관한 적용례) 제156조의2제5항 본문, 제156조의4제2항 본문 및 제156조의6제4항 본문의 개정규정은 이 법 시행 당시 각각 같은 개정규정에 따른 경정청구기간이 만료되지 아니한 경우에도 적용한다.\n\n제10조(외국인 통합계좌 원천징수 특례에 관한 적용례) 제156조의9의 개정규정은 이 법 시행 이후 외국인 통합계좌의 명의인에게 국내원천소득을 지급하는 경우부터 적용한다.\n\n제11조(국외 주식매수선택권등 거래명세서 제출에 관한 적용례) 제164조의5의 개정규정은 이 법 시행 이후 임원등이 주식매수선택권을 행사하거나 주식기준보상을 지급받는 경우부터 적용한다.\n\n제12조(신탁소득에 대한 소득세 과세에 관한 경과조치) 이 법 시행 전에 신탁재산에 귀속된 소득에 대해서는 제2조의3제2항의 개정규정에도 불구하고 종전의 규정에 따른다.\n\n제13조(분리과세 기준금액 상향에 관한 경과조치) 이 법 시행 전에 지급받은 연금소득의 분리과세 기준금액에 관하여는 제14조제3항제9호다목의 개정규정에도 불구하고 종전의 규정에 따른다.\n\n제14조(배당가산율 인하에 관한 경과조치) 이 법 시행 전에 지급받은 배당소득의 소득금액 계산에 관하여는 제17조제3항 각 호 외의 부분 단서의 개정규정에도 불구하고 종전의 규정에 따른다.\n\n제15조(주택의 보증금등에 대한 총수입금액 계산의 특례에 관한 경과조치) 이